In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/109_2/NLP/Final_Report

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
 
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text
 
def clean_doc(doc):
	tokens = doc.split()
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	tokens = [word for word in tokens if word.isalpha()]
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
def add_doc_to_vocab(filename, vocab):
	doc = load_doc(filename)
	tokens = clean_doc(doc)
	vocab.update(tokens)

def process_docs(path, vocab):
	add_doc_to_vocab(path, vocab)

vocab = Counter()
process_docs('txt_sentoken/neg.txt', vocab)
process_docs('txt_sentoken/pos.txt', vocab)
print(len(vocab))
print(vocab.most_common(50))

In [ ]:
def save_list(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_list(vocab, 'vocab.txt')

In [ ]:
from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

def clean_doc(doc, vocab):
	tokens = doc.split()
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	tokens = [w for w in tokens if w in vocab]
	tokens = ' '.join(tokens)
	return tokens

def process_docs(path, vocab):
  documents = list()
  file = open(path, 'r')
  doc = file.readline()
  while doc:
    tokens = clean_doc(doc, vocab)
    documents.append(tokens)
    doc = file.readline()
  file.close()
  return documents

vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

all_positive_docs = process_docs('txt_sentoken/pos.txt', vocab)
all_negative_docs = process_docs('txt_sentoken/neg.txt', vocab)
positive_docs = all_positive_docs[:2500]
negative_docs = all_negative_docs[:2500]
train_docs = negative_docs + positive_docs
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define training labels
ytrain = array([0 for _ in range(len(negative_docs))] + [1 for _ in range(len(positive_docs))])

positive_docs = all_positive_docs[2500:]
negative_docs = all_negative_docs[2500:]
test_docs = negative_docs + positive_docs
encoded_docs = tokenizer.texts_to_sequences(test_docs)
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
ytest = array([0 for _ in range(len(negative_docs))] + [1 for _ in range(len(positive_docs))])

def process_docs(path, vocab):
  documents = list()
  file = open(path, 'r')
  doc = file.readline()
  while doc:
    tokens = clean_doc(doc, vocab)
    documents.append(tokens)
    doc = file.readline()
  file.close()
  return documents

eval_docs = process_docs('txt_sentoken/eval.txt', vocab)
encoded_eval_docs = tokenizer.texts_to_sequences(eval_docs)
Xeval = pad_sequences(encoded_eval_docs, maxlen=max_length, padding='post')

vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=6, verbose=2)
loss, acc = model.evaluate(Xtest, ytest, verbose=0)

print('Test Accuracy: %f' % (acc*100))

In [ ]:
results = (model.predict(Xeval) > 0.5).astype("int32")
with open("results.txt", "w", encoding="utf-8") as f:
  for i in range(len(results)):
    if results[i][0] == 0:
      f.writelines("fake\n")
    else:
      f.writelines("real\n")